# DQN(Deep Q Learning)(RL)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import gym

BATCH_SIZE = 32
LR = 0.01
EPSILON = 0.9
GAMMA = 0.9
TARGET_REPLACE_ITER = 100
MEMORY_CAPACITY = 2000
env = gym.make('CartPole-v0')  # 通过 OpenAI Gym 创建一个名为 'CartPole-v0' 的环境实例
env = env.unwrapped# 取消环境的限制，使其可以进行更多的操作
N_ACTIONS = env.action_space.n
N_STATES = env.observation_space.shape[0]

env.unwrapped：比如原始环境可能对步数有最大限制（如 200 步），解锁后可以通过自定义配置改变这个限制。  
env.action_space.n：获取该环境中动作空间的维度（即动作的数量）。对于 CartPole-v0，动作空间是离散的，只有两个动作：0 表示将小车向左推；1 表示将小车向右推。因此，这里 N_ACTIONS 的值为 2  
env.observation_space.shape[0]：获取该环境中状态空间的维度（即状态特征的数量）；CartPole-v0包含小车的位置；小车的速度；杆子的角度；杆子的角速度。

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(N_STATES, 10)#输入环境状态，输出隐藏层
        self.fc1.weight.data.normal_(0, 0.1)
        self.out = nn.Linear(10, N_ACTIONS)#输出小车要采取的动作的价值
        self.out.weight.data.normal_(0, 0.1)

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        actions_value = self.out(x)
        return actions_value
    #其实就是
    #def forward(self, x):
        # y = self.fc1(x)
        # z = F.relu(y)
        # actions_value = self.out(z)
        # return actions_value

使用正态分布（高斯分布）随机初始化第一层全连接层（fc1）的权重：  
self.fc1：表示网络中的第一层全连接层。  
self.fc1.weight：表示这一层的权重。  
self.fc1.weight.data：表示获取这一层权重的底层数据。  
.normal_(0, 0.1)：表示用均值为 0，标准差为 0.1 的正态分布随机填充数据。  

初始化的权重是什么：nn.Linear 层的 W 矩阵

In [ ]:
class DQN(object):
    def __init__(self):
        self.eval_net,self.target_net = Net(),Net()#这是两个一模一样的网络
        self.memory = np.zeros((MEMORY_CAPACITY, N_STATES * 2 + 2))  # 经验回放记忆
        self.memory_counter = 0
        self.learn_step_counter = 0
        self.optimizer = torch.optim.Adam(self.eval_net.parameters(), lr=LR)
        self.loss_func = nn.MSELoss()

    def choose_action(self, x):
        #随机生成一个浮点数,默认范围[0, 1),用来决定是否选择当前的最优动作（利用）或随机动作（探索）。
        if np.random.uniform() < EPSILON:  #greedy
            actions_value = self.eval_net(x)
            action = torch.max(actions_value, 1)[1].data.numpy()[0,0] #[0,0]：提取二维数组中的具体值
            
        else: #随机选择动作
            action = np.random.randint(0, N_ACTIONS)
        return action

    #经验回放存储机制
    def store_transition(self, s, a, r, s_):  #（当前状态，执行的动作，即使奖励，下一状态）
    #s(), a标量（0或1）, r标量, s_()
        transition = np.hstack((s, a, r, s_))  # 合并成一个一维数组:若 s 和 s_ 的维度为 4，a 为标量，r 为标量,那么transition 的形状为 (4,1,1,4,)
        index = self.memory_counter % MEMORY_CAPACITY
        self.memory[index, :] = transition  # 经验池 self.memory 的第 index 行存储当前的 transition
        self.memory_counter += 1  # 每存储一条新经验后，计数器 self.memory_counter 增加 1
        
    def learn(self):
        #target_net的更新    不一步一更新
        if self.learn_step_counter % TARGET_REPLACE_ITER == 0:  # 每隔 TARGET_REPLACE_ITER 步更新一次目标网络
            self.target_net.load_state_dict(self.eval_net.state_dict())#将评估网络的参数复制到目标网络中，以更新目标网络的参数
            #state_dict() 方法来获取模型的所有参数和缓冲区，本例中是nn.Linear 层的权重和偏置

***EPSILON:*** ε-贪心策略中的超参数；在 ε-贪心策略中，智能体会以概率 EPSILON 选择当前的最优动作（即利用策略），以概率 1 - EPSILON 随机选择一个动作（即探索新动作）；  
作用：控制探索和利用的平衡，较大的 EPSILON 值，意味着智能体更倾向于选择当前的最优动作（利用）；较小的 EPSILON 值，意味着智能体更倾向于随机探索新动作（探索）。  

***GAMMA:*** 折扣因子，决定智能体对未来奖励的重视程度。  
强化学习中，目标是最大化未来的累积奖励。折扣因子用于控制未来奖励的重要性：  

$$Q(s,a)=r+γ⋅maxQ(s′,a′)$$

Q(s,a):状态s下执行动作a所能获得的长期累积奖励的期望  
r:当前奖励  
γ:折扣因子
maxQ(s′,a′):下一状态的最大 Q 值  
解释：当 GAMMA 接近 1 时，智能体会更加重视长期奖励；当 GAMMA 接近 0 时，智能体会更加重视当前的即时奖励。  

***MEMORY_CAPACITY:*** 是经验池的容量，表示最多可以存储多少条经验

两个网络一模一样：  
self.eval_net（评价网络）：用来计算当前状态x下所有可能的动作的预测的 Q 值  
每一步都在更新  
self.target_net（目标网络）：  
定期更新，由TARGET_REPLACE_ITER决定，详见def learn(self):

解释 $action = torch.max(actions\_value, 1)[1].data.numpy()[0,0]:$   
torch.max(input, dim) 用于在指定维度上，计算输入张量的最大值及其索引，返回一个元组 (values, indices)(最大值，索引)  
[1] 是对 torch.max(actions_value, 1) 返回的元组 (values, indices) 中的第二个元素（indices）进行索引操作。也就是说，[1] 提取的是每一行最大值的索引。

#### shape的dim：  
$actions_value = torch.tensor([[0.1, 0.5, 0.3],
                              [0.8, 0.2, 0.4]])$  
| Row (Batch)   | Action 1 | Action 2 | Action 3 |
|---------------|----------|----------|----------|
| Sample 1      | 0.1      | 0.5      | 0.3      |
| Sample 2      | 0.8      | 0.2      | 0.4      |

dim=0：表示按 列 计算最大值，即跨行（纵向）比较每一列的值。  
dim=1：表示按 行 计算最大值，即跨列（横向）比较每一行的值。  
那么 dim=1 表示我们希望“从每一行中取列的最大值”：
第一行 [0.1, 0.5, 0.3] 中，最大值是 0.5，对应的列索引为 1。
第二行 [0.8, 0.2, 0.4] 中，最大值是 0.8，对应的列索引为 0。

actions_value的shape是(batch_size,num_actions)，也就是包含了前批次中所有状态下的所有动作分别对应的价值  
本例中每一次训练都包含四个状态（小车的位置、小车的速度、杆子的角度、杆子的角速度），即actions_value包含了：只考虑小车位置时0，1动作对应的价值；只考虑小车速度时……

$def store_transition(self, s, a, r, s_):$  
self.memory_counter经验计数器，在它小于MEMORY_CAPACITY时，index会因为取余操作而随memory_counter递增，然后将经验值transition存入第index行，最后计数+1。  
当计数大于MEMORY_CAPACITY时，index重新遍历0~MEMORY_CAPACITY-1,覆盖掉self.memory中的旧数据，所以说MEMORY_CAPACITY是经验池的流量。  
为什么不设置成∞：内存有限。